In [123]:
# Import Module
import numpy as np
import folium as fo
import pandas as pd
import matplotlib.pyplot as plt
from folium.plugins import FloatImage

In [124]:
# Cleaning Data
web = pd.read_html('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Indonesia')
df = web[3]
df.columns=["Province","Confirmed","Recovered","Deaths","Active","Website","nav"]
df = df.drop(["nav","Website"],axis=1)
df = df.drop([34,35,36])

In [127]:
df[["Confirmed","Recovered","Deaths","Active"]] = df[["Confirmed","Recovered","Deaths","Active"]].astype("int32")
df["Province"][12]="Jakarta"

C:\Users\bima\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [129]:
# input data geo
geo = pd.read_json("https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json")

In [131]:
# merge dataframe
geo=geo.rename({"Provinsi":"Province"},axis=1)
dfM = df.merge(right=geo)
dfM["Cor"]=list(map(lambda x,y:[x,y],dfM["latitude"],dfM["longitude"]))

In [142]:
dfM.sample(5)

,Province,Confirmed,Recovered,Deaths,Active,latitude,longitude,logo,Cor
11,Gorontalo,0,0,0,0,0.699937,122.446724,https://3.bp.blogspot.com/-75ec3fJKhgU/UNl0Tdg...,"[0.6999371999999999, 122.4467238]"
33,Yogyakarta,22,1,2,19,-7.875385,110.426209,https://upload.wikimedia.org/wikipedia/commons...,"[-7.8753849, 110.4262088]"
1,Bali,10,0,2,8,-8.409518,115.188916,https://4.bp.blogspot.com/-ELlrLdH0frM/WSz4Ajq...,"[-8.4095178, 115.188916]"
27,West Java,149,6,19,124,-7.090911,107.668887,https://3.bp.blogspot.com/-BRV8kAVyi4I/XDX3S86...,"[-7.090911, 107.668887]"
6,Central Kalimantan,7,0,0,7,-1.681488,113.382355,https://upload.wikimedia.org/wikipedia/commons...,"[-1.6814878, 113.3823545]"


In [144]:
# create map
peta = fo.Map(
    location=[-0.789275,113.9213257],
    zoom_start=5
)
# create marker
for i in dfM.index:
    fo.Marker(location=dfM["Cor"][i],
              popup=(
                  f'<b>Total={dfM["Active"][i]} \n Sembuh={dfM["Recovered"][i]}\n Meninggal={dfM["Deaths"][i]} </b>'),
              tooltip=dfM["Province"][i],
              icon=fo.Icon(color="red")
             ).add_to(peta)
peta.add_child(fo.LatLngPopup())
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Peta Persebaran COVID-19</b></h3>
             '''
peta.get_root().html.add_child(fo.Element(title_html))

peta

In [146]:
peta.save('covid_19.html')